In [1]:
import notebook_common
import jaconv
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    r_word_2400_xlsx,
    w_word_2400_csv,
    w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    # w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    pattern_table_spoken,
    pattern_td_spoken,
    r_spoken_freq_nlb_with_enter_txt,
    w_spoken_freq_nlb_with_enter_csv,
    w_spoken_freq_nlb_with_enter_filtered_csv,
    w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    w_spoken_freq_nlb_with_enter_filtered_hr_csv,
    w_spoken_freq_nlb_sumed_csv,
    w_tmp_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
)

p("Import common module")

Import common module


In [2]:
# 格式化后的的口语词频文件
lst = []
with open(r_spoken_freq_nlb_with_enter_txt, "r", encoding="utf-8") as f:
    content = f.read()  # 一次性读取
    table_matches = pattern_table_spoken.findall(content)  # 或 finditer 迭代匹配
    for table_match in table_matches:
        td_matches_lst = pattern_td_spoken.findall(table_match)
        lst.append(td_matches_lst)
    columns = ["headword", "type", "reading", "freq", "freq_per", "rank"]
    df = pd.DataFrame(lst, columns=columns)
    # 将次数映射到新列
    df["cnt_head"] = df.groupby(["headword"]).transform("size")
    df["cnt_read"] = df.groupby(["reading"]).transform("size")
    df["cnt_head_read"] = df.groupby(["headword", "reading"]).transform("size")
    to_csv_sig(df, w_spoken_freq_nlb_with_enter_csv)
p("w_spoken_freq_nlb_with_enter_csv--ok")

w_spoken_freq_nlb_with_enter_csv--ok


In [3]:
df

,headword,type,reading,freq,freq_per,rank,cnt_head,cnt_read,cnt_head_read
0,.,記号,．,"989,922",8.7065,0.99900,1,2,1
1,(,記号,（,"851,151",7.4860,0.99890,1,2,1
2,*,*,"329,229",2.8956,0.99728,None,265,1,1
3,*,記号,＊,"306,861",2.6989,0.99710,265,2,1
4,/,記号,／,"199,654",1.7560,0.99524,1,2,1
...,...,...,...,...,...,...,...,...,...
98173,｝,記号,｝,"3,281",0.0289,0.86545,1,2,1
98174,～,記号,～,"152,926",1.3450,0.99366,1,1,1
98175,￡,記号,ポンド,82,0.0007,0.46012,1,1,1
98176,￣,記号,￣,804,0.0071,0.72208,1,2,1


In [4]:
df.dtypes

headword         object
type             object
reading          object
freq             object
freq_per         object
rank             object
cnt_head          int64
cnt_read          int64
cnt_head_read     int64
dtype: object

In [5]:
# 指定列中，只要有一列的数据为空，删除该行,Inplace=True
na_columns = ["headword", "type", "reading", "freq", "rank"]
df.dropna(subset=na_columns, inplace=True)
# 将词频转化为数字
df["freq"] = pd.to_numeric(df["freq"].str.replace(",", ""), errors="coerce")
# 将记号和*号相关记录删除
mask = (df["type"] == "記号") | (df["headword"] == "*")
# 将发音reading转化为平假名
df["hira"] = df["reading"].apply(jaconv.kata2hira)
df2 = df[~mask].copy()
df2.sort_values(by="freq", ascending=False, inplace=True)
to_csv_sig(df2, w_spoken_freq_nlb_with_enter_filtered_csv)

p("w_spoken_freq_nlb_with_enter_filtered_csv--ok")

w_spoken_freq_nlb_with_enter_filtered_csv--ok


In [6]:
# 处理汇总数据
df_spoken_sumed = df2["freq"].groupby([df["headword"], df["hira"]]).sum().to_frame()
df_spoken_sumed.rename(columns={"freq": "spoken_freq"}, inplace=True)
df_spoken_sumed.sort_values(by="spoken_freq", ascending=False, inplace=True)
to_csv_sig(df_spoken_sumed, w_spoken_freq_nlb_sumed_csv, True)
p(df_spoken_sumed)
p("w_spoken_freq_nlb_sumed_csv--ok")

                    spoken_freq
headword hira                  
の        の             57221425
に        に             35286930
を        を             33707106
は        は             31279365
が        が             30705135
...                         ...
ｋワット     きろわっと                1
右顧       うこ                   1
右脳する     うのうする                1
ｃｍする     せんちめーとるする            1
主役する     しゅやくする               1

[96948 rows x 1 columns]
w_spoken_freq_nlb_sumed_csv--ok


In [7]:
df_spoken_sumed.index

MultiIndex([(     'の',      'ノ'),
            (     'に',      'ニ'),
            (     'を',      'ヲ'),
            (     'は',      'ハ'),
            (     'が',      'ガ'),
            (     'て',      'テ'),
            (     'た',      'タ'),
            (    'ます',     'マス'),
            (     'だ',     'ダロ'),
            (     'と',      'ト'),
            ...
            (  '遠謀する', 'エンボウスル'),
            ('ちびちびする', 'チビチビスル'),
            (  'ちばする',   'チバスル'),
            (   '遠海魚', 'エンカイギョ'),
            (    '遡江',    'ソコウ'),
            (  '遡求する', 'ソキュウスル'),
            (   '遠駆け',   'トオガケ'),
            (  '遠近する', 'オチコチスル'),
            (  'ちさする',   'チサスル'),
            ( '咲き賑わう', 'サキニギワウ')],
           names=['headword', 'reading'], length=96949)